In [72]:
import os
import json
import enum
import warnings
from typing import List, Dict, Any
from sqlalchemy import create_engine, MetaData
from pydantic import BaseModel

from dotenv import load_dotenv
load_dotenv(override=True)
warnings.filterwarnings("ignore")

# LangChain / LangGraph
from langchain.prompts.prompt import PromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# SQL
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit

# Qdrant
from qdrant_client import QdrantClient, models

# Optional search tool (kept but unused in graph)
from langchain_community.tools import BraveSearch

In [73]:
BRAVE_API_KEY = os.getenv("BRAVE_API_KEY")
POSTGRES_URI = os.getenv("POSTGRES_URI")
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_COLLECTION_NAME = os.getenv("QDRANT_COLLECTION_NAME")
LLM_MODEL = os.getenv("LLM_MODEL")
EMBEDED_MODEL_NAME = os.getenv("EMBEDED_MODEL_NAME")
# ────────────────────────────────────────────────────────────────────────────────
# LLMs & DB
llm = ChatOllama(model=LLM_MODEL, temperature=0.1)
emb_model = OllamaEmbeddings(model=EMBEDED_MODEL_NAME)
engine = create_engine(POSTGRES_URI)
db = SQLDatabase(engine=engine)

def get_sql_tools(db: SQLDatabase, llm: ChatOllama):
    """Create SQL tools for ReAct agent."""
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    return toolkit.get_tools()

DB_TOOLS = get_sql_tools(db, llm)


In [74]:
class Tools(enum.Enum):
    DOCUMENT = "Document"
    SUBMITTAL = "Submittal"
    RFI = "RFI"
    INSPECTION = "Inspection"
    UNKNOWN = "Unknown"
    KANBAN = "Kanban"
    DIRECTORY = "Directory"
    TRANSMITTAL = "Transmittal"
    WORK_ORDER = "Work Order"
    SAFETY = "Safety"
    INSPECTION_TEST_PLAN = "Inspection Test Plan"
    SPECIFICATION = "Specification"
    SCHEDULE = "Schedule"
    FORM = "Form"
    LOCATION = "Location"
    TAG = "Tag"
    WORK_ORDER_GROUP = "Work Order Group"
    MEETING = "Meeting"

class RoutingDecision(BaseModel):
    question: str
    tool: Tools
    reasoning: str

# Create Router Agent

In [75]:
prompt_router = """
Analyze the user query below and determine its Available Tools.
Available Tools (Choose one):
- Document: For questions about user's documents related to the project that they uploaded to the system. It's not related to attachmented submittal files or other.
- Submittal: For questions about construction submittals. 
- RFI: For questions about construction requests for information (RFI).
- Inspection: For questions about construction inspections.
- Work Order: For questions about construction work orders.
- Unknown: If the user's query is not related to any of the above features. Try to answer the user's question as best you can.

Query: {question}
"""

In [76]:
user_query = {
    "question": "Hello",
    #"Get me a latest rejected work order",
    "user_id": 1,
    "project_id": 1,
    "company_id": 1,
}

In [77]:
# Structed Output follow RoutingDecision schema
llm_router = llm.with_structured_output(RoutingDecision)

response_router = llm_router.invoke(prompt_router.format(question=user_query["question"]))

print("Question: ", response_router.question)
print("features: ", response_router.tool)
print("Reasoning: ", response_router.reasoning)

Question:  Hello
features:  Tools.UNKNOWN
Reasoning:  The query is a greeting and not related to any of the available tools. The available tools are for questions about documents, construction submittals, RFI, inspection, work orders, or unknown. The query does not relate to any of these categories.


# Check Permission

User ห้ามถามเกี่ยวกับ Feature นั้น ๆ permission_id < 1 (0=Not Allowed)

- Step 1: User ask question
- Step 2: Analyze question is related to Avalable Features?
- Step 3: If UNKNOWN, its mean question is not related to any features
- Step 3: Check if permission_id >= 1

## Query Database

In [78]:
from sqlalchemy import MetaData, Table, select, bindparam

metadata = MetaData()

a   = Table("auth_user", metadata, schema="public", autoload_with=engine)
cu  = Table("company_companyuser", metadata, schema="public", autoload_with=engine)
c   = Table("company_company", metadata, schema="public", autoload_with=engine)
pu  = Table("project_projectuser", metadata, schema="public", autoload_with=engine)
p   = Table("project_project", metadata, schema="public", autoload_with=engine)
cp  = Table("company_permission", metadata, schema="public", autoload_with=engine)
cpg = Table("company_permissiongroup", metadata, schema="public", autoload_with=engine)
tl  = Table("company_toollabels", metadata, schema="public", autoload_with=engine)

stmt = (
    select(tl.c.title).distinct()
    .select_from(
        a
        .join(cu, a.c.id == cu.c.user_id)
        .join(c, c.c.id == cu.c.company_id)
        .join(pu, a.c.id == pu.c.user_id)
        .join(p, p.c.id == pu.c.project_id)
        .join(cp, cp.c.id == pu.c.permission_id)
        .join(cpg, cpg.c.permission_id == pu.c.permission_id)
        .join(tl, tl.c.id == cpg.c.tool_id)
    )
    .where(
        a.c.id == bindparam("user_id"),
        c.c.id == bindparam("company_id"),
        p.c.id == bindparam("project_id"),
        tl.c.title == bindparam("tool_title"),
    )
)

In [79]:

params = {
    "user_id": user_query["user_id"],
    "company_id": user_query["company_id"],
    "project_id": user_query["project_id"],
    "tool_title": "response_router.features.value",
}

is_valid_permission = False
with engine.connect() as conn:
    res = conn.execute(stmt, params).scalars().all()
    if res:
        is_valid_permission = True

if is_valid_permission:
    print("is_valid_permission: ", is_valid_permission)
else:
    print("No results found. User does not have permission.")

# To inspect the exact SQL with literals:
print(stmt.compile(engine, compile_kwargs={"literal_binds": True}))


No results found. User does not have permission.
SELECT DISTINCT public.company_toollabels.title 
FROM public.auth_user JOIN public.company_companyuser ON public.auth_user.id = public.company_companyuser.user_id JOIN public.company_company ON public.company_company.id = public.company_companyuser.company_id JOIN public.project_projectuser ON public.auth_user.id = public.project_projectuser.user_id JOIN public.project_project ON public.project_project.id = public.project_projectuser.project_id JOIN public.company_permission ON public.company_permission.id = public.project_projectuser.permission_id JOIN public.company_permissiongroup ON public.company_permissiongroup.permission_id = public.project_projectuser.permission_id JOIN public.company_toollabels ON public.company_toollabels.id = public.company_permissiongroup.tool_id 
WHERE public.auth_user.id = NULL AND public.company_company.id = NULL AND public.project_project.id = NULL AND public.company_toollabels.title = NULL


## If UNKNOWN and not valid permission

In [81]:
# If UNKNOWN and not valid permission

prompt_template = """
You are a helpdesk for a construction company.
You given a question and 2 different reasons why you CANNOT answer the question.
1. Its not related to any of the available features and the reason is: {reason}
2. Its not valid permission: tool permission={permission}

question: {question}
"""


if response_router.tool == Tools.UNKNOWN or not is_valid_permission:
    # prompt = PromptTemplate(
    #     input_variables=["question"],
    #     template=prompt_template,
    # )
    response = llm.invoke(prompt_template.format(reason=response_router.reasoning, permission=is_valid_permission, question=user_query["question"]))
    print("Response: ", response.content)

else:
    print("Response: ", response_router.reasoning)

Response:  <think>
Okay, the user asked "Hello" and I need to check if there are any reasons why I can't answer it. The first reason says it's not related to the available tools. The tools mentioned are documents, construction submittals, RFI, inspection, work orders, or unknown. "Hello" doesn't fit any of those. So that's valid. The second reason is about tool permission being False, but that's probably not applicable here. So the answer should be straightforward, pointing out that the query is a greeting and not related to any of the provided tools.
</think>

The query is a greeting and not related to any of the available tools.


In [8]:
# def check_permissions(user_id: str, company_id: str, project_id: str, feature: AvailableFeatures) -> bool:

# Router

# Retrieval

# Text-to-SQL

# Inference